In [8]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import cmasher as cmr
import numpy as np
import math
from db_connect.utils import DbConnect

In [9]:
HOST = "local.data-workshop.workshops.utkusarioglu.com"
PORT = 5432
DB_NAME = "happiness"
USER = "happiness_reader"
PASSWORD = "happiness_reader"

db = DbConnect(HOST, PORT, DB_NAME, USER, PASSWORD)

In [10]:
changes = db.query("SELECT * FROM raw.highest_and_lowest_scores")
changes

,country_or_region,highest_year,highest_score,high_duplicity,lowest_year,lowest_score,low_duplicity,change_direction,change
0,Afghanistan,2017,3.79,1,2019,3.20,1,decrease,0.59
1,Albania,2015,4.96,1,2018,4.59,1,decrease,0.37
2,Algeria,2016,6.36,1,2019,5.21,1,decrease,1.15
3,Angola,2015,4.03,1,2017,3.80,2,decrease,0.23
4,Argentina,2016,6.65,1,2019,6.09,1,decrease,0.56
...,...,...,...,...,...,...,...,...,...
165,Venezuela,2015,6.81,1,2019,4.71,1,decrease,2.10
166,Vietnam,2015,5.36,1,2016,5.06,1,decrease,0.30
167,Yemen,2015,4.08,1,2018,3.36,1,decrease,0.72
168,Zambia,2015,5.13,1,2019,4.11,1,decrease,1.02


In [11]:
all_raw = db.query("SELECT * FROM raw.years_all")
all = all_raw.loc[:, ~all_raw.columns.isin(["id", "region"])]
all

,year,country_or_region,happiness_rank,happiness_score,standard_error,lower_confidence_interval,upper_confidence_interval,gdp_per_capita,family,life_expectancy,freedom,government_corruption,generosity,dystopia_residual
0,2015,Haiti,119,4.52,0.07,NaN,NaN,0.27,0.74,0.39,0.24,0.17,0.46,2.24
1,2016,Yemen,147,3.72,NaN,3.62,3.83,0.58,0.47,0.31,0.23,0.06,0.10,1.97
2,2016,Cameroon,114,4.51,NaN,4.42,4.61,0.52,0.63,0.13,0.43,0.06,0.23,2.52
3,2019,Singapore,34,6.26,NaN,NaN,NaN,1.57,1.46,1.14,0.56,0.27,0.45,NaN
4,2015,Bahrain,49,5.96,0.05,NaN,NaN,1.32,1.22,0.75,0.45,0.31,0.17,1.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,2017,Panama,30,6.45,NaN,6.35,6.56,1.23,1.37,0.71,0.55,0.07,0.21,2.31
778,2016,Israel,11,7.27,NaN,7.20,7.34,1.34,1.00,0.85,0.36,0.09,0.32,3.31
779,2016,Poland,57,5.84,NaN,5.75,5.92,1.25,1.05,0.69,0.45,0.06,0.14,2.20
780,2015,Chad,149,3.67,0.04,NaN,NaN,0.34,0.76,0.15,0.24,0.05,0.18,1.94


In [12]:
usa = all[all["country_or_region"] == "United States"]
usa_gdp_per_year = usa[["year", "gdp_per_capita"]]
usa_gdp_per_year

,year,gdp_per_capita
38,2015,1.39
231,2016,1.51
454,2019,1.43
582,2018,1.40
706,2017,1.55


In [13]:
all.loc[
    all["country_or_region"].isin(["Afghanistan", "United States", "Turkey"]), :
].groupby("country_or_region").mean()

,year,happiness_rank,happiness_score,standard_error,lower_confidence_interval,upper_confidence_interval,gdp_per_capita,family,life_expectancy,freedom,government_corruption,generosity,dystopia_residual
country_or_region,,,,,,,,,,,,,
Afghanistan,2017.0,149.4,3.512,0.03,3.500,3.650,0.356,0.410,0.254,0.118,0.116,0.212,2.083333
Turkey,2017.0,75.2,5.414,0.04,5.355,5.535,1.150,1.182,0.704,0.258,0.114,0.088,2.086667
United States,2017.0,15.8,6.998,0.04,6.965,7.130,1.456,1.330,0.820,0.508,0.204,0.292,2.486667


In [14]:
def richestAndPoorest(df: pd.DataFrame, ind: pd.Index, col: str) -> str:
    val = df[col].loc[ind]
    if val > 2 or val < 1:
        return "Richest and Poorest"
    return "Mid Income"


all.groupby(lambda ind: richestAndPoorest(all, ind, "gdp_per_capita")).mean(
    numeric_only=True
)

,year,happiness_rank,happiness_score,standard_error,lower_confidence_interval,upper_confidence_interval,gdp_per_capita,family,life_expectancy,freedom,government_corruption,generosity,dystopia_residual
Mid Income,2016.997354,47.243386,6.151164,0.044925,5.985444,6.166746,1.257540,1.245265,0.779762,0.452460,0.165238,0.200582,2.125339
Richest and Poorest,2016.990099,108.128713,4.657450,0.050330,4.423427,4.640420,0.596683,0.922450,0.456683,0.372921,0.144010,0.180372,2.059872
